# Indice proyecto Predictor creditos bancarios:
* **0- Datos + EDA Credit failure prediction**
  * Link: https://colab.research.google.com/drive/1OEB_xe0QN__ABY8uueQnofnQszqlZkP2?usp=sharing

* **1-Limpieza Datos y Feature Credit failure prediction**
    * Link:https://colab.research.google.com/drive/1z-CUbk-R5nvjH9H4Dz-1Ti8CUVsvbQsS?usp=sharing
    
**Nota**: Al inicio del proyecto se  guardo el archivo original con nombre *cs-training.csv y cs-test.csv* en local y se continúa trabajando con la copia del archivo llamada *cs-training.csv.*

# Predicción de Impagos Hipotecarios con Machine Learning

### Objetivo del Proyecto

El objetivo de este proyecto es desarrollar un modelo predictivo que permita identificar posibles impagos en créditos hipotecarios, utilizando técnicas avanzadas de Machine Learning.

Aunque se trata de un caso simulado, los datos provienen del conjunto **"Give Me Some Credit"** publicado en [Kaggle](https://www.kaggle.com/c/GiveMeSomeCredit), ampliamente utilizado para problemas de scoring crediticio.

Este dataset permite replicar un escenario realista del sector financiero y practicar con un flujo completo de modelado: desde la limpieza de datos hasta la puesta en producción.

Se ha aplicado un enfoque integral con balanceo de clases, selección de modelos basados en boosting y ensamblado de algoritmos para mejorar la capacidad predictiva.


### Enfoque y Técnicas Aplicadas

- **Limpieza estructurada de datos** mediante un `Pipeline` modular.
- **Exploración de correlaciones** para comprender las variables más relevantes.
- **Balanceo de clases** con **SMOTE** ante un fuerte desbalance en la variable objetivo.
- **Modelado con algoritmos de boosting**: `XGBoost`, `LightGBM` y `CatBoost`.
- **Optimización automática de hiperparámetros** con `Optuna` para afinar el rendimiento.
- **Modelo ensamblado** mediante `VotingClassifier` para combinar fortalezas de distintos algoritmos.
- **Ajuste del umbral de decisión** para mejorar el trade-off entre `precision` y `recall`.

### Evaluación y Métricas
Los modelos fueron evaluados con métricas clave:

- `ROC AUC`
- `Precision`
- `Recall`
- `F1-score`

Esto permitió ajustar el modelo a objetivos específicos del negocio, como **minimizar falsos negativos** (clientes morosos no detectados) sin penalizar excesivamente la precisión.

### Evaluación y Métricas

Los modelos han sido evaluados con las siguientes métricas clave:

| **Modelo**                                         | **Precision** | **Recall** | **F1-score** | **ROC AUC** |
|----------------------------------------------------|--------------|-----------|-------------|------------|
| VotingClassifier (XGB + CatBoost, umbral = 0.57)  | 0.3229       | 0.5922    | 0.4179      | 0.8591     |
| CatBoost                                         | 0.3834       | 0.4873    | 0.4291      | 0.8585     |
| XGBoost                                          | 0.2673       | 0.6677    | 0.3818      | 0.8570     |
| LightGBM                                         | 0.3383       | 0.5362    | 0.4148      | 0.8549     |

El modelo **VotingClassifier (XGB + CatBoost, umbral = 0.57)** ha sido seleccionado para guardar, ya que presenta **el mejor desempeño en términos de ROC AUC (0.8591)** y mantiene un **buen equilibrio entre precisión y recall**. Este enfoque garantiza la correcta identificación de clientes morosos, minimizando falsos negativos sin sacrificar precisión.

In [ ]:
#@title **1.Importación de librerías para análisis, modelos y métricas Credit failure prediction**

#Instalar librerías Boosting (XGBoost, LightGBM y CatBoost)
!pip install -q pandas numpy scikit-learn xgboost lightgbm catboost imbalanced-learn shap matplotlib seaborn joblib

#Instalar optuna
!pip install optuna catboost xgboost lightgbm --quiet

# Manipulación de datos
import pandas as pd  # Procesar, analizar y limpiar datos
import numpy as np  # Operaciones matemáticas y manipulación de matrices

# Visualización de datos
import seaborn as sns  # Visualización de gráficos estadísticos
import matplotlib.pyplot as plt  # Generación de gráficos

# Preprocesamiento y escalado
from sklearn.model_selection import train_test_split  # División en Train/Test
from sklearn.preprocessing import StandardScaler, RobustScaler  # Normalización y escalado
from sklearn.compose import ColumnTransformer  # Transformaciones por columnas
from sklearn.base import BaseEstimator, TransformerMixin # Creación de transformadores personalizados
import optuna # Optimización de hiperparámetros

# Modelos de clasificación (Boosting)
from xgboost import XGBClassifier  # Boosting optimizado
from lightgbm import LGBMClassifier  # Boosting eficiente en grandes datos
from catboost import CatBoostClassifier  # Boosting robusto para datos categóricos

# Evaluación de modelos
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, classification_report
)
from sklearn.ensemble import VotingClassifier # Combinación de modelos

# Balanceo de datos desbalanceados
from imblearn.over_sampling import SMOTE  # Técnica para generar datos sintéticos en clases minoritarias

# Interpretabilidad del modelo
import shap  # Explicación de impacto de características en modelos

# Gestión de almacenamiento
import joblib  # Guardado y carga de modelos entrenados

print("Librerías cargadas correctamente.")

Librerías cargadas correctamente.


In [ ]:
#@title 2.Importar csv para limpieza
from limpieza_basica import LimpiezaBasica
df = pd.read_csv("cs-training.csv")
df_limpio = LimpiezaBasica().fit_transform(df)


In [ ]:
#@title 3.Division datos X - Y
from division_datos import dividir_datos

X_train, X_test, y_train, y_test = dividir_datos(df_limpio)


División de datos completada:
Train: 119385 muestras
Test:  29847 muestras


In [ ]:
#@title 4.Preprocesamiento: Winsorización + Escalado + SMOTE
from preprocesamiento import preprocesar_datos

columnas_winsor = ['Edad', 'UsoCrédito', 'IngresoMensual', 'RatioDeuda',
                   '30-59DíasTarde', '60-89DíasTarde', '90DíasTarde',
                   'PréstamosCasa', 'LíneasCrédito', 'Dependientes']

X_train_final, y_train_final, X_test_scaled, scaler = preprocesar_datos(X_train, X_test, y_train, columnas_winsor)



Preprocesamiento completado:
X_train_final shape: (222764, 10)
y_train_final: [111382 111382]
X_test shape: (29847, 10)


#Boosting Models

In [ ]:
#@title Instalar librerías Boosting (XGBoost, LightGBM y CatBoost)

#!pip install -q xgboost lightgbm catboost

In [ ]:
#@title 5.Definir modelos XGBC, LGBMC, CatBoost
#import numpy as np
#import pandas as pd
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
#from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Definir modelos con parámetros optimizados
modelos = {
    'XGBoost': XGBClassifier(
        n_estimators=250, learning_rate=0.03, max_depth=3,  # Ajustamos profundidad y estabilidad
        min_child_weight=10, gamma=2,  # Reducimos falsos positivos
        eval_metric='error', scale_pos_weight=1.5,  # Ajustamos peso de clases
        tree_method='hist', predictor='cpu_predictor',  # Configuración para CPU
        use_label_encoder=False, verbosity=0, random_state=42
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=200, learning_rate=0.05, max_depth=4,  # Mantiene estabilidad
        num_leaves=50, min_data_in_leaf=50, min_child_samples=30,  # Ajuste para evitar sobreajuste
        class_weight='balanced', device='cpu', verbosity=-1, random_state=42
    ),
    'CatBoost': CatBoostClassifier(
        iterations=200, learning_rate=0.05, depth=4, verbose=0, random_state=42  # Configuración óptima
    )
}

# Entrenar y evaluar cada modelo
resultados = []
for nombre, modelo in modelos.items():
    modelo.fit(X_train_final, y_train_final)  # Entrenamiento

    y_proba = modelo.predict_proba(X_test_scaled)[:, 1]  # Probabilidades clase positiva
    threshold = 0.6 if nombre == "XGBoost" else 0.5  # Ajuste de umbral solo para XGBoost
    y_pred = (y_proba >= threshold).astype(int)

    # Calcular métricas de evaluación
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    # Guardar resultados
    resultados.append({
        "Modelo": nombre,
        "Precision": round(precision, 4),
        "Recall": round(recall, 4),
        "F1-score": round(f1, 4),
        "ROC AUC": round(auc, 4)
    })

# Mostrar ranking de resultados ordenados por ROC AUC
df_resultados = pd.DataFrame(resultados).sort_values(by='ROC AUC', ascending=False).reset_index(drop=True)
display(df_resultados)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


,Modelo,Precision,Recall,F1-score,ROC AUC
0,CatBoost,0.3834,0.4873,0.4291,0.8585
1,XGBoost,0.2673,0.6677,0.3818,0.8570
2,LightGBM,0.3383,0.5362,0.4148,0.8549


In [ ]:
#@title 6.Código completo: Optuna + VotingClassifier + Evaluación Soft
# 1. Instalación
#!pip install optuna catboost xgboost lightgbm --quiet

# 2. Imports
#import optuna
#from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
#from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import VotingClassifier
#import pandas as pd

# 3. Reducir conjunto para optimización (más rápido)
X_train_opt, _, y_train_opt, _ = train_test_split(X_train_final, y_train_final, train_size=0.5, random_state=42)

# 4. Función Optuna
def objective(trial, model_name):
    if model_name == "XGBoost":
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 100, 300),
            "max_depth": trial.suggest_int("max_depth", 3, 6),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 3.0),
            "eval_metric": "auc",
            "use_label_encoder": False,
            "random_state": 42,
            "verbosity": 0
        }
        model = XGBClassifier(**params)

    elif model_name == "CatBoost":
        params = {
            "iterations": trial.suggest_int("iterations", 100, 300),
            "depth": trial.suggest_int("depth", 3, 6),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 5),
            "random_state": 42,
            "verbose": 0
        }
        model = CatBoostClassifier(**params)

    model.fit(X_train_opt, y_train_opt)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    return roc_auc_score(y_test, y_proba)

# 5. Optimización
studies = {}
for modelo in ["XGBoost", "CatBoost"]:
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective(trial, modelo), n_trials=25, n_jobs=-1)
    studies[modelo] = study
    print(f"\n Mejor configuración para {modelo}:")
    print(study.best_params)

# 6. Recuperar mejores parámetros
best_params_xgb = studies["XGBoost"].best_params
best_params_cat = studies["CatBoost"].best_params

# 7. VotingClassifier con mejores parámetros
voting_model = VotingClassifier(
    estimators=[
        ('xgb', XGBClassifier(**best_params_xgb, eval_metric='auc', use_label_encoder=False, random_state=42)),
        ('cat', CatBoostClassifier(**best_params_cat, verbose=0, random_state=42))
    ],
    voting='soft' # Cambiar a 'hard' si deseas votación dura
)

voting_model.fit(X_train_final, y_train_final)

# 8. Evaluar con umbral ajustado
threshold = 0.57  # puedes cambiar este valor si lo deseas
y_proba = voting_model.predict_proba(X_test_scaled)[:, 1]
y_pred = (y_proba >= threshold).astype(int)

# 9. Métricas
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"\n VotingClassifier (XGB + CatBoost, umbral = {threshold})")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")
print(f"ROC AUC:   {roc_auc:.4f}")


[I 2025-06-13 17:10:56,931] A new study created in memory with name: no-name-7e476a1f-7c9f-4b2d-9795-33420ff458d9
[I 2025-06-13 17:10:59,371] Trial 0 finished with value: 0.8512189624501116 and parameters: {'n_estimators': 146, 'max_depth': 5, 'learning_rate': 0.1298589749956598, 'subsample': 0.8455770830840823, 'colsample_bytree': 0.6858909738227577, 'scale_pos_weight': 1.8924174274409686}. Best is trial 0 with value: 0.8512189624501116.
[I 2025-06-13 17:11:00,245] Trial 1 finished with value: 0.8466533270748812 and parameters: {'n_estimators': 169, 'max_depth': 6, 'learning_rate': 0.16112229175158937, 'subsample': 0.5152301369820504, 'colsample_bytree': 0.7616109028516169, 'scale_pos_weight': 1.4202274890220903}. Best is trial 0 with value: 0.8512189624501116.
[I 2025-06-13 17:11:02,697] Trial 2 finished with value: 0.8510637305063623 and parameters: {'n_estimators': 237, 'max_depth': 4, 'learning_rate': 0.14209120730509384, 'subsample': 0.6796398117474061, 'colsample_bytree': 0.8329


 Mejor configuración para XGBoost:
{'n_estimators': 296, 'max_depth': 3, 'learning_rate': 0.02661366768786684, 'subsample': 0.7609672697528725, 'colsample_bytree': 0.9959093213796765, 'scale_pos_weight': 2.407447743340021}


[I 2025-06-13 17:11:54,306] Trial 0 finished with value: 0.8567181341455969 and parameters: {'iterations': 177, 'depth': 3, 'learning_rate': 0.10814758421980175, 'l2_leaf_reg': 2.23555036222491}. Best is trial 0 with value: 0.8567181341455969.
[I 2025-06-13 17:12:01,026] Trial 1 finished with value: 0.8559697024288259 and parameters: {'iterations': 218, 'depth': 4, 'learning_rate': 0.010291662938737806, 'l2_leaf_reg': 2.2269766798100608}. Best is trial 0 with value: 0.8567181341455969.
[I 2025-06-13 17:12:02,651] Trial 2 finished with value: 0.8579564451775405 and parameters: {'iterations': 165, 'depth': 5, 'learning_rate': 0.0490797588431715, 'l2_leaf_reg': 3.3824925888880086}. Best is trial 2 with value: 0.8579564451775405.
[I 2025-06-13 17:12:16,845] Trial 4 finished with value: 0.8570499997433589 and parameters: {'iterations': 224, 'depth': 4, 'learning_rate': 0.0907299573505088, 'l2_leaf_reg': 1.8209003717786887}. Best is trial 2 with value: 0.8579564451775405.
[I 2025-06-13 17:12


 Mejor configuración para CatBoost:
{'iterations': 245, 'depth': 6, 'learning_rate': 0.03212054034693248, 'l2_leaf_reg': 4.943499801394491}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [17:14:45] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 VotingClassifier (XGB + CatBoost, umbral = 0.57)
Precision: 0.3229
Recall:    0.5922
F1-score:  0.4179
ROC AUC:   0.8591


In [ ]:
#@title 7.Guardar el modelo final
import joblib

# Guardar el modelo final VotingClassifier
joblib.dump(voting_model, "modelo_voting.pkl")

# Guardar el escalador RobustScaler si fue utilizado
joblib.dump(scaler, "scaler_robust.pkl")

print("Modelo y escalador guardados correctamente en el entorno de trabajo.")


Modelo y escalador guardados correctamente en el entorno de trabajo.
